# Biosolids data filtering 

Author: Sahar H. El Abbadi

Date: March 20, 2024

Goal: to clean biosolids data, removing permits that are likely not associated with wastewater treatment plants. 


In [ ]:
# Setup

import pandas as pd 
import pathlib 
from utilities import check_all_sic_code # for generating SIC codes for each NPDES permit 
from utilities import check_for_ww_permits # check SIC code and classify as "sewer_system" or "other_system" 
from tqdm import tqdm 
import pandoc

tqdm.pandas() # for progress bars in df.progress_applhy 


## Generate SIC biosolids dataset 

1. Load biosolids dataset downloaded by Christina. Saved in 02_raw_data as Data_Download_1699657092121.csv
2. For each NPDES ID, look up all associated SIC codes. 
3. Save dataframe as pickle file and as CSV 

In [ ]:
# Load raw data and generate datasets 

### ALERT: this takes 4+ hours to run. Comment it out and load pickle file as needed. 

# all_biosolids = pd.read_csv(pathlib.PurePath('01_raw_data', 'Data_Download_1699657092121.csv'))
# 
# # test on top row 
# # all_biosolids = all_biosolids.head(2).copy()
# all_biosolids['sic_permit'] = all_biosolids['NPDES ID'].progress_apply(check_all_sic_code)
# all_biosolids.to_pickle(pathlib.PurePath('05_pickle_files', 'biosolids_data_sic_codes.pkl'))
# all_biosolids.to_csv((pathlib.PurePath('04_results', 'biosolids_data_sic_codes.csv')))


In [ ]:
# Load pickle file 
all_biosolids = pd.read_pickle(pathlib.PurePath('05_pickle_files', 'biosolids_data_sic_codes.pkl'))

## List of facilities to remove 

Generate a list of facilities to remove, using the following filtering criteria:

1. Does the facility have a sewer-related SIC code? If yes --> keep 
2. Is the facility listed as a POTW under its reporting obligations? If yes --> keep 
3. Of remaining facilities, check SIC codes. If NO MATCH --> keep. 
4. Of now remaining facilities with a non-sewer SIC match, manually keep or remove based on SIC codes 

In [ ]:
# Apply filter based on SIC sewer related code 
# This takes ~15 minutes to run. Load pickle to save time 
all_biosolids['check_sewer_permits'] = all_biosolids['NPDES ID'].progress_apply(check_for_ww_permits)
biosolids_to_remove = all_biosolids[all_biosolids['check_sewer_permits'] == 'other_system']
biosolids_to_remove.to_pickle(pathlib.PurePath('05_pickle_files', 'biosolids_data_sic_codes_not_sewer.pkl'))


In [ ]:
# Load pickle for biosolids that have already been filtered based on whether or not they have a sewer-related code 
biosolids_not_sewer = pd.read_pickle(pathlib.PurePath('05_pickle_files', 'biosolids_data_sic_codes_not_sewer.pkl'))


In [ ]:
# Apply filter based on POTW reporting obligation 

potw_mask = ~biosolids_not_sewer['Reporting Obligation(s)'].str.contains('POTW')
biosolids_not_sewer_not_potw = biosolids_not_sewer[potw_mask]

display(biosolids_not_sewer_not_potw[['Facility Name', 'NPDES ID', 'sic_permit', 'check_sewer_permits']])

In [29]:
# Keep facilities that have no match (ie remove them from our list of facilities to remove) 

# These facilities have a match with an SIC code that is NOT sewer-related
biosolids_not_sewer_not_potw_has_match = biosolids_not_sewer_not_potw[biosolids_not_sewer_not_potw['sic_permit'].apply(lambda x: 'NO_SIC_MATCH' not in x)]
print(f'Length of dataframe (not sewer, not POTW, has SIC match): {len(biosolids_not_sewer_not_potw_has_match)}')


Length of dataframe (not sewer, not POTW, has SIC match): 226


In [ ]:
# Check how many facilities have no match after previous filtering 

biosolids_not_sewer_not_potw_no_match = biosolids_not_sewer_not_potw[biosolids_not_sewer_not_potw['sic_permit'].apply(lambda x: 'NO_SIC_MATCH' in x)]

print(f'Length of dataframe (not sewer, not POTW, no SIC match): {len(biosolids_not_sewer_not_potw_no_match)}')

### Remove facilities with problematic SIC codes 

Remove facilities with SIC codes that are unlikely to be associated with publicly owned wastewater treatment facilities

In [30]:
# Apply filter based on reporting obligation 
sic_remove = [6515, # mobile homes
              4941, # water supply
              8211, # schools
              8221, # colleges & universities
              7033, # trailer parks / campsites 
              7032, # sporting and recreation camps 
              9223, # correctional facilities 
              1389, # oil & gas field services 
              3533, # oil and gas field machinery 
              8361, # residential care 
              8661, # religious orgs 
              7997, # sports / recreation clubs
              7999, # amusement and recreation 
              8051, # skilled nursing care 
              3498, # fabricated pipe & fitting 
              7011, # hotels and motels 
              3171, # handbags & purses 
              2491, # wood preserving 
              2493, # reconsistuted wood products
              9711, # national security 
              3743, # railroad equipment 
              5541, # gas station services 
              4911, # electric services 
              5075, # heating & cooling 
              7041, # membership hotels 
              2011, # meat packing plants
              8063, # psychiatric hospitals 
              5812, # eating places 
              7999, # amusement parks
              2899, # chemical preparation (spice / food extraction)
              3331, # primary copper
              6531, # real estate agents & managers
              4011, # railroads
              6514, # dwelling operators (residential)
              2621, # paper mills
              4581, # airports
              1522, # residential construction 
              ]

sic_check = [1629, # heavy construction 
             9511, # air, water, solid waste management
             9199, # general government  
             7299, # misc. personal services 
             2819, # 
             ]

# Check SIC codes for facilities that have an SIC code match 
biosolids_not_sewer_not_potw_has_match_sic_removal = biosolids_not_sewer_not_potw_has_match[biosolids_not_sewer_not_potw_has_match['sic_permit'].apply(lambda x: any(item in sic_remove for item in x))]
display(biosolids_not_sewer_not_potw_has_match_sic_removal[['Facility Name', 'NPDES ID', 'sic_permit', 'check_sewer_permits']])


,Facility Name,NPDES ID,sic_permit,check_sewer_permits
14,AUSTIN COUNTY WSC PLANT 3,TX0125709,"[4941, 4941]",other_system
15,LAKE PFLUGERVILLE WWTF,TX0132721,"[1541, 4941, 4941]",other_system
16,PURTIS CREEK STATE PARK WWTP,TX0082856,"[7033, 7033]",other_system
25,CHISOS BASIN WWTP,TX0094684,"[7999, 7999]",other_system
35,LAUGHLIN AFB WWTP BLDG 1004,TX0022608,"[9711, 1542, 9711, 9711, 1542, 4581, 9711, 971...",other_system
...,...,...,...,...
3189,ATK LAUNCH SYSTEMS INC,UTL024805,"[7549, 3714, 3769, 3714, 3761, 3769, 3764, 754...",other_system
3800,OAKELY CITY,UTL020061,"[2899, 2899]",other_system
3867,"KENNECOTT UTAH COPPER, LLC",UTL000051,"[3331, 3331, 1021, 3331, 1021]",other_system
3870,LYSTEK INTERNATIONAL,CAL000001,"[7538, 7538, 4212, 4212, 7513, 7513, 8211, 399...",other_system


In [ ]:
biosolids_not_sewer_not_potw_has_match_sic_check = biosolids_not_sewer_not_potw_has_match[~biosolids_not_sewer_not_potw_has_match['sic_permit'].apply(lambda x: any(item in sic_remove for item in x))]
display(biosolids_not_sewer_not_potw_has_match_sic_check[['Facility Name', 'NPDES ID', 'sic_permit', 'check_sewer_permits']])

### Manually check the remaining facilities 

The dataset of biosolids_not_sewer_not_potw_has_sic_check contains the facilities that I'm not confident removing based solely on their SIC codes. Manually inspect facilities and decide where they should be kept based on name / information available online 

1. Live Oak County Safety Rest Area WWTF - code 7299 (misc personal services) --> remove, rest area along highway 
2. Bayou Club WWTP - code 8641 (civic & social associations) --> remove, dining club 
3. GE Packaged Power Jport - code 3511 (turbines / turbine generators), 7699 (repair services) --> probably remove, GE and not public 
4. Sigmapro WWTP - code 6519 (real property lessors) --> Sigma Pro private company WWTP, not public 
5. US DOE / Savannah River Site - codes 2819 (industrial inorganic chemicals), 9611 (administration of general economic programs) --> remove, not a public wastewater treatment facility 

Based on this online search, we can remove all facilities in the original subset biosolids_not_sewer_not_potw_has_match (before filtering based on the specific SIC codes of concern). 


In [ ]:
biosolids_to_remove = biosolids_not_sewer_not_potw_has_match

biosolids_to_remove.to_csv(pathlib.PurePath('04_results', 'biosolids_to_remove.csv'), index=False)
biosolids_to_remove.to_pickle(pathlib.PurePath('05_pickle_files', 'biosolids_to_remove.pkl'))

## Check the POTW facilities for water treatment codes

It occurred to me after completing this analysis that there might be drinking water treatment plants in facilities that consider themselves to be POTWs 


In [ ]:
# Data exploration 

from utilities import summarize_sic_codes

biosolids_not_sewer = pd.read_pickle(pathlib.PurePath('05_pickle_files', 'biosolids_data_sic_codes_not_sewer.pkl'))
# Apply filter based on POTW reporting obligation 

potw_mask = biosolids_not_sewer['Reporting Obligation(s)'].str.contains('POTW')
biosolids_not_sewer_is_potw = biosolids_not_sewer[potw_mask]

display(biosolids_not_sewer_is_potw[['Facility Name', 'NPDES ID', 'sic_permit', 'check_sewer_permits']])

# summarize_sic_codes(biosolids_not_sewer_is_potw, 'sic_permit').to_csv(pathlib.PurePath('04_results', 'biosolids_not_sewer_is_potw_summary.csv'))

In [31]:
# Run code to test 

biosolids_not_sewer = pd.read_pickle(pathlib.PurePath('05_pickle_files', 'biosolids_data_sic_codes_not_sewer.pkl'))

# Make columns indicating if the facility lists POTW for reporting obligation 
biosolids_not_sewer['potw_check'] = biosolids_not_sewer['Reporting Obligation(s)'].str.contains('POTW')

# Make a column indicating if facility has a DW code 
dw_code = 4941
biosolids_not_sewer['dw_sic_check'] = biosolids_not_sewer['sic_permit'].apply(lambda x: dw_code in x)

# Keep facilities where potw_check == True and dw_sic_check == False 
potw_not_dw_mask = ~((biosolids_not_sewer['potw_check'] == 1) & (biosolids_not_sewer['dw_sic_check'] == 0))
biosolids_not_potw_includes_dw = biosolids_not_sewer[potw_not_dw_mask]

# Keep facilities that have no match (ie remove them from our list of facilities to remove) 

# These facilities have a match with an SIC code that is NOT sewer-related
biosolids_not_sewer_not_potw_has_match = biosolids_not_potw_includes_dw[biosolids_not_potw_includes_dw['sic_permit'].apply(lambda x: 'NO_SIC_MATCH' not in x)]
print(f'Length of dataframe (not sewer, not POTW (unless has DW permit), has SIC match): {len(biosolids_not_sewer_not_potw_has_match)}')

# Check SIC codes for facilities that have an SIC code match 
biosolids_not_sewer_not_potw_has_match_sic_removal = biosolids_not_sewer_not_potw_has_match[biosolids_not_sewer_not_potw_has_match['sic_permit'].apply(lambda x: any(item in sic_remove for item in x))]
display(biosolids_not_sewer_not_potw_has_match_sic_removal[['Facility Name', 'NPDES ID', 'sic_permit', 'check_sewer_permits']])

Length of dataframe (not sewer, not POTW (unless has DW permit), has SIC match): 246


,Facility Name,NPDES ID,sic_permit,check_sewer_permits
14,AUSTIN COUNTY WSC PLANT 3,TX0125709,"[4941, 4941]",other_system
15,LAKE PFLUGERVILLE WWTF,TX0132721,"[1541, 4941, 4941]",other_system
16,PURTIS CREEK STATE PARK WWTP,TX0082856,"[7033, 7033]",other_system
25,CHISOS BASIN WWTP,TX0094684,"[7999, 7999]",other_system
35,LAUGHLIN AFB WWTP BLDG 1004,TX0022608,"[9711, 1542, 9711, 9711, 1542, 4581, 9711, 971...",other_system
...,...,...,...,...
3800,OAKELY CITY,UTL020061,"[2899, 2899]",other_system
3867,"KENNECOTT UTAH COPPER, LLC",UTL000051,"[3331, 3331, 1021, 3331, 1021]",other_system
3870,LYSTEK INTERNATIONAL,CAL000001,"[7538, 7538, 4212, 4212, 7513, 7513, 8211, 399...",other_system
4135,BLUE SKY RANCH AND RESORT,UTL025763,"[7011, 7011]",other_system


In [32]:
biosolids_not_sewer_not_potw_has_match_sic_check = biosolids_not_sewer_not_potw_has_match[~biosolids_not_sewer_not_potw_has_match['sic_permit'].apply(lambda x: any(item in sic_remove for item in x))]
display(biosolids_not_sewer_not_potw_has_match_sic_check[['Facility Name', 'NPDES ID', 'sic_permit', 'check_sewer_permits']])

,Facility Name,NPDES ID,sic_permit,check_sewer_permits
554,LIVE OAK COUNTY SAFETY REST AREA WWTF,TX0129321,"[7299, 7299]",other_system
938,BAYOU CLUB WWTP,TX0083933,[8641],other_system
1082,GE PACKAGED POWER JPORT,TX0101656,"[3511, 3511, 7699, 3511, 7699, 3511, 7699]",other_system
1462,SIGMAPRO WWTP,TX0138754,[6519],other_system
2856,US DOE/SAVANNAH RIVER SITE,SCL000175,"[2819, 2819, 9611, 2819, 2819, 2819, 2819]",other_system


In [33]:
import datetime

biosolids_to_remove = biosolids_not_sewer_not_potw_has_match

current_datetime = datetime.datetime.now()

# Format the datetime as a string to append to the file name
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M")

biosolids_to_remove.to_csv(pathlib.PurePath('04_results', f'biosolids_to_remove_{formatted_datetime}.csv'), index=False)
biosolids_to_remove.to_pickle(pathlib.PurePath('05_pickle_files', f'biosolids_to_remove_{formatted_datetime}.pkl'))

In [53]:
# Final comparison of the two methods 

remove_original = pd.read_pickle(pathlib.PurePath('05_pickle_files', 'biosolids_to_remove.pkl'))
remove_update = pd.read_pickle(pathlib.PurePath('05_pickle_files', 'biosolids_to_remove_2024-03-25_11-32.pkl'))

# common columns only 
common_columns = ['Facility Name', 'NPDES ID', 'City', 'State', 'Reporting Obligation(s)']

# Identify indices of differences based on common columns
compare_removal_list_indices = pd.concat([remove_original[common_columns], remove_update[common_columns]]).drop_duplicates(keep=False).index

# Add 'sic_permit' column from remove_update for facilities only in remove_update
compare_removal_list = remove_update.loc[compare_removal_list_indices, ['Facility Name', 'NPDES ID', 'City', 'State', 'Reporting Obligation(s)', 'sic_permit']]


print(compare_removal_list)
print(f'{len(compare_removal_list)} additional facilities to remove')
# Now I want to add the column ['sic_permit'] to compare_removal_list. Because these facilities are only in remove_update, I need to match them appropriately to remove_update


                                         Facility Name   NPDES ID  \
462                            TEASELVILLE WELL 1 WWTF  TX0118273   
891                                       TURLOCK WQCF  CAL078948   
1684                                         CANAL WTP  TX0135500   
1991                                        ONEIDA (C)  NYL026956   
2040                                     PORTLAND WWTP  INL020095   
2120                                   NEW WINDSOR (T)  NYL022446   
2190                                    JEFFERSON WWTF  TX0002542   
2219                                  POUGHKEEPSIE (T)  NYL026271   
2227                               WESTMINSTER CITY OF  COL024171   
2243                                UC DAVIS MAIN WWTP  CAL077895   
2665                         BARDSTOWN TOWN CREEK WWTP  KYL020237   
2707                                BOONVILLE, CITY OF  MOL040738   
2771  INDIAN RIVER COUNTY UTILITIES DEPARTMENT (IRCUD)  FLL037940   
2903                              